# Lexer do Python

Nesta aula iremos acessar o lexer do Python e reaproveitá-lo em outros projetos. O Python expõe tanto o seu lexer quanto o seu parser como bibliotecas incluídas na distribuição padrão. Iremos integrar o lexer do Python ao ox e, a partir daí, começar a utilizá-lo como lexer genérico em nossos projetos.

Para começar, considere uma string de código Python:

In [1]:
st = "print('hello world!')"

A tokenização esta string deve gerar quatro tokens: **print**, **(**, **'hello world'** e **)**. Vamos importar o lexer do Python para gerar nossa função de tokenização

In [7]:
import tokenize

Tokenizamos o código através da função `tokenize.tokenize`. Esta função possui uma interface curiosa. Ao invés de pedir uma string de código, ela requer uma função que, cada vez que for chamada, retorna uma nova linha do código como bytes. Vamos fazer uma função que recebe uma string de código e retorna uma função deste tipo

Podemos ver a documentação com o comando

In [ ]:
tokenize.tokenize?

Agora o nosso line getter:

In [18]:
def make_line_getter(code):
    lines = code.splitlines()
    
    def getter():
        if lines:
            # pop(0) retira e retorna o primeiro elemento da lista 
            line = lines.pop(0)
            
            # Agora convertemos para bytestring
            return line.encode('utf-8')
        
        # Se não houverem mais linhas, retorna uma bytestring vazia
        else:
            return b''
        
    return getter

Vamos imprimir todas tokens:

In [21]:
for tok in tokenize.tokenize(make_line_getter(st)):
    print(tok)

TokenInfo(type=59 (ENCODING), string='utf-8', start=(0, 0), end=(0, 0), line='')
TokenInfo(type=1 (NAME), string='print', start=(1, 0), end=(1, 5), line="print('hello world!')")
TokenInfo(type=53 (OP), string='(', start=(1, 5), end=(1, 6), line="print('hello world!')")
TokenInfo(type=3 (STRING), string="'hello world!'", start=(1, 6), end=(1, 20), line="print('hello world!')")
TokenInfo(type=53 (OP), string=')', start=(1, 20), end=(1, 21), line="print('hello world!')")
TokenInfo(type=0 (ENDMARKER), string='', start=(2, 0), end=(2, 0), line='')


Observe que o tokenizer retorna objetos do tipo TokenInfo, enquanto o ox espera objetos 
do tipo ox.Token. Vamos fazer a conversão entre ambos. Existem algumas diferenças importantes entre as duas classes e é necessário fazer algumas conversões manuais.

In [28]:
# dicionário que guarda um mapa entre o código dos tokens e os
# respectivos nomes.
TYPE_CODE_TO_NAME = {}
TYPE_NAME_TO_CODE = {}

def to_ox_token(tok):
    type_no = tok.type
    value = tok.string
    lineno = tok.start[0]
    lexpos = tok.start[1]
    type_str = TYPE_CODE_TO_NAME[type_no]
    return Token(type_str, value, lineno, lexpos)

O nosso principal trabalho é determinar a conversão de código numérico usado pelo TokenInfo para uma string, como utiliza o ox.Token. Felizmente o módulo tokenize possui uma série de constantes que fazem estas conversões. Por exemplo, tokenize.NAME é uma constante com o valor 1, que corresponde ao código numérico para tokens do tipo NAME.

Vamos utilizar a introspecção do Python para montar este dicionário.

In [29]:
namespace = vars(tokenize)  # converte o módulo para um dicionário

for name, value in namespace.items():
    # consideramos que as constantes de tipo possuem nomes em letras 
    # maiúsculas e valores numéricos
    if name.isupper() and isinstance(value, int):
        print(name, value)
        TYPE_CODE_TO_NAME[value] = name
        TYPE_NAME_TO_CODE[name] = value

ENDMARKER 0
NAME 1
NUMBER 2
STRING 3
NEWLINE 4
INDENT 5
DEDENT 6
LPAR 7
RPAR 8
LSQB 9
RSQB 10
COLON 11
COMMA 12
SEMI 13
PLUS 14
MINUS 15
STAR 16
SLASH 17
VBAR 18
AMPER 19
LESS 20
GREATER 21
EQUAL 22
DOT 23
PERCENT 24
LBRACE 25
RBRACE 26
EQEQUAL 27
NOTEQUAL 28
LESSEQUAL 29
GREATEREQUAL 30
TILDE 31
CIRCUMFLEX 32
LEFTSHIFT 33
RIGHTSHIFT 34
DOUBLESTAR 35
PLUSEQUAL 36
MINEQUAL 37
STAREQUAL 38
SLASHEQUAL 39
PERCENTEQUAL 40
AMPEREQUAL 41
VBAREQUAL 42
CIRCUMFLEXEQUAL 43
LEFTSHIFTEQUAL 44
RIGHTSHIFTEQUAL 45
DOUBLESTAREQUAL 46
DOUBLESLASH 47
DOUBLESLASHEQUAL 48
AT 49
ATEQUAL 50
RARROW 51
ELLIPSIS 52
OP 53
AWAIT 54
ASYNC 55
ERRORTOKEN 56
N_TOKENS 60
NT_OFFSET 256
COMMENT 57
NL 58
ENCODING 59


Agora vamos criar uma função que junta estas funcionalidades e gera tokens ox a partir do código Python

In [30]:
def python_tokenize(source):
    getter = make_line_getter(source)
    tokens = tokenize.tokenize(getter)
    ox_tokens = [to_ox_token(tok) for tok in tokens]
    return ox_tokens

In [31]:
python_tokenize(st)

[ENCODING(utf-8),
 NAME(print),
 OP((),
 STRING('hello world!'),
 OP()),
 ENDMARKER()]

Observe que o Python cria 2 tokens "virtuais" que não existem originalmente no código: um token inicial de ENCODING e outro de ENDMARKER. Na realidade, o tokenizador do Python faz isto em outras situações: temos os tokens INDENT e DEDENT para fazer o controle de indentação e isto livra esta tarefa complicada do parser.

In [35]:
st = """
while True:
    x += 1
"""
python_tokenize(st)

[ENCODING(utf-8),
 NAME(while),
 NAME(True),
 OP(:),
 INDENT(    ),
 NAME(x),
 OP(+=),
 NUMBER(1),
 DEDENT(),
 ENDMARKER()]

## Exercício: Pytuguês

Com o lexer do Python, podemos criar versões alternativas do Python quem simplesmente modificam algumas palavras chave. Um uso interessante para isto é realizar traduções do Python para outras linguagens

Faça uma versão do "Pytuguês", traduzindo o Python para português. 

1. Tokenize com o tokenizador do Python
2. Traduza alguns tokens especiais como (se -> if, else -> senão, while -> enquanto, etc).
3. Gere o código Python a partir da lista de tokens modificada.
4. Porque a abordagem por tokenização é melhor que uma simples substituição de strings?